In [1]:
import matplotlib as plt 
import numpy as np 
from matplotlib import colors
from PIL import Image, ImageDraw
import random
import pandas as pd
from collections import defaultdict
import random

In [2]:
def createList(r1, r2):#option creation for random edge addition
        if (r1 == r2):
            return r1
        else:
            res = []
            while(r1 != r2):
                res.append(r1)
                r1 += 1
                if (r1 > 50):
                    r1 = r1-50
            res.append(r2)
        return res 

In [3]:
class Graph():
    
    def __init__(self):
        
        #create graph
        self.graph = {}
        for i in range(1,51):
            if i-1 == 0:
                self.graph[i] = [2, 50]
            elif i+1 == 51:
                self.graph[i] = [1, 49]
            else:
                self.graph[i] = [i+1, i-1]
        
        #create list of nodes
        self.nodes = createList(1, 50)
                
    def addEdge(self):
        boo1 = True
        while boo1:
            if len(self.nodes) == 0:
                boo1 = False
                break
            val = random.choice(self.nodes)
            self.nodes.remove(val)
            valstart = val-5
            valend = val+5
            if (valstart < 1):
                valstart = valstart+50
            if (valend > 50):
                valend = valend-50
            val_nodes = createList(valstart, valend)
            if(val == 1): #deleting adjacent options so that double edges arent made
                val_nodes.remove(50)
            elif(val == 50):
                val_nodes.remove(1)
            else: 
                val_nodes.remove(val-1)
                val_nodes.remove(val+1)
            boo2 = False
            while len(val_nodes) > 0:
                newnode = random.choice(val_nodes)
                if newnode in self.nodes:
                    self.nodes.remove(newnode)
                    self.graph[val].append(newnode)
                    self.graph[newnode].append(val)
                    boo2 = True
                    break
                else:
                    val_nodes.remove(newnode)
                    
            if boo2:
                break
        if boo1 == False:
            return False
        else:
            return True

In [4]:
#start position, end position, backwards path to take
#returns path
def BFS(start, end, graph):
 
    # a queue of the vertices whose path is to be scanned
    # boolean array for checking if node has been visited at least once or not
    # all moves in path set to -1
    queue = []
    visited = [False for i in range(51)]
    back_path = [-1 for i in range(51)]
    
    # start is visited and added to the queue
    visited[start] = True
    queue.append(start)

    # BFS algorithm - look at all neighbors first
    while (len(queue) != 0):
        u = queue[0]
        queue.pop(0)
        for i in range(len(graph[u])):
            if (visited[graph[u][i]] == False):
                visited[graph[u][i]] = True
                back_path[graph[u][i]] = u
                queue.append(graph[u][i])

                # We stop BFS when we find
                # destination.
                if (graph[u][i] == end):
                    path = []
                    crawl = end
                    path.append(crawl)
                    
                    #reversing back_path and removing -1 from nodes that are not in the path to get full path
                    while (back_path[crawl] != -1):
                        path.insert(0, back_path[crawl])
                        crawl = back_path[crawl]
                    return path

    return [end]

In [5]:
class Predator():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure that predator doesn't start where the agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
    
    def pred_distracted_move(self, g, agentpos): #predator moving with probability 0.6 to node closer to agent and probability 0.4 to any other random neighbour
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths 
        new_path = random.randint(0,(len(shortest_path)-1))
        chance = random.randint(1,100)
        if chance <= 60: #60 percent chance of moving correctly
            self.pos = shortest_path[new_path][0]
        else: #40 percent chance of moving in distracted manner
            self.pos = random.choice(neighbors)
        return self
    
    def probability_distracted(self, g, ):
        

In [6]:
class Prey():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure prey doesn't start where agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def prey_move(self, g):
        prey_loc_neighbours = [self.pos]
        for i in g.graph[self.pos]:
            prey_loc_neighbours.append(i)
        #randomly choose neighbor to move to (or staying put)
        self.pos = random.choice(prey_loc_neighbours)
        return self

In [ ]:
class Utility:
    def __init__(self):
        self.Ustar = np.zeros((51, 51, 51)) # Initialize UStar map
    
    def action_value(self, g, agentpos, preyneighbors, predneighbors, predator):
        future_utility = 0
        for preypos in pr_pos_list:
            for predpos in pd_pos_list:
                # figure out distracted probability
                probability = (1.0/len(preyneighbors))*((0.4/len(predneighbors)) + predator.probability_distracted(env, pd, pd_pos, ag_pos))
                future_utility += probability*(1 + self.Ustar[agentpos][preypos][predpos])
        return future_utility

    def value_iteration(self, g):
        
        #initialize UStar
        for agentpos in range(1, 51): 
              for preypos in range(1, 51):
                for predpos in range(1, 51):
                    if (agentpos == preypos) and (agentpos != predpos):
                        self.Ustar[agentpos][preypos][predpos] = 0
                    elif (agentpos == predpos) or (abs(agentpos - predpos) == 1):
                        self.Ustar[agentpos][preypos][predpos] = 1000
                    else:
                        self.Ustar[agentpos][preypos][predpos] = 1

        #Update UStar based on the likely movements of the prey and predator
        while True:
            currUstar = np.zeros((51, 51, 51))
            for agentpos in range(1, 51): 
                for preypos in range(1, 51):
                    for predpos in range(1, 51):
                        if (agentpos == preypos) and (agentpos != predpos):
                            currUstar[agentpos][preypos][predpos] = 0
                        elif (agentpos == predpos) or (abs(agentpos - predpos) == 1):
                            currUstar[agentpos][preypos][predpos] = 1000
                        elif (abs(agentpos - preypos) == 1):
                            currUstar[agentpos][preypos][predpos] = 1
                        else:
                            action_values = []
                            for move in g[agentpos]:
                                # Run loop until value converges
                                preyneighbors = [preypos]
                                predneighbors = []
                                for neighbor in g[preypos]:
                                    preyneighbors.append(neighbor)
                                for neighbor in g[predpos]:
                                    predneighbors.append(neighbor)
                            currval = self.action_value(g, move, preyneighbors, predneighbors, predpos)
                            action_values.append(currval)
            
            currUstar[agentpos][preypos][predpos] = min(action_values) 
      

            max_error = 0
            for ag_pos in range(1, 51): 
                for pr_pos in range(1, 51):
                    for pd_pos in range(1, 51):
                        max_error = max(max_error, abs(self.Ustar[agentpos][preypos][predpos] - currUstar[agentpos][preypos][predpos]))
                        print('error', max_error)            
            if max_error < 0.001:
                break
            self.Ustar = currUstar
        

In [ ]:
#based on agent 2 of the previous project
def AgentUStar():
    
    
    